In [39]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import requests
import json

In [14]:
# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(model_name='bert-base-multilingual-cased')

# Load the title vector store from the local directory
title_vector_store = FAISS.load_local('title_faiss_index', embeddings, allow_dangerous_deserialization=True)

No sentence-transformers model found with name bert-base-multilingual-cased. Creating a new one with mean pooling.


In [23]:
query = "Как подписать контракт после отказа заказчика от заключения?"

In [27]:
# Retrieve the top 3 most relevant titles
top_k = 10
relevant_titles = title_vector_store.similarity_search(query, k=top_k)

In [28]:
# Extract the article titles from the results
relevant_topic_names = [doc.page_content for doc in relevant_titles]

In [29]:
# Print the relevant topic names
print("Relevant topic names:")
for topic in relevant_topic_names:
    print(f"- {topic}")

Relevant topic names:
- Как выбрать другого поставщика по закупки по потребности, если поставщик отказался от заключения контракта?
- Подписание контракта после отказа от заключения со стороны заказчика.
- Почему не получается скачать документ контракта/подписанный документ контракта?
- Можно ли изменить номер контракта в карточке контракта?
- Что делать, если не отображаются подписи в заключенном контракте?
- Как заполнить раздел «Контактная информация» в профиле компании в случае отсутствия сайта?
- Как заполнить раздел «Контактная информация» в профиле компании в случае отсутствия сайта?
- Как проверить причину отклонения заявки на регистрацию/изменение данных компании на Портале поставщиков?
- Как на Портале поставщиков ознакомиться с реестром заказчиков?
- Как в УПД по контракту, имеющему электронное исполнение через ГИС ЕИС, заполнить строку 5а?


In [30]:
def retrieve_relevant_chunks(query, relevant_topic_names, chunk_vector_store, top_k=5):
    """
    Retrieve the most relevant chunks from the descriptions of the relevant articles.

    Args:
        query (str): The user's query.
        relevant_topic_names (list): List of relevant article titles from the first step.
        chunk_vector_store (FAISS): The vector store with description chunks.
        top_k (int): Number of top chunks to retrieve.

    Returns:
        list: List of relevant chunk texts.
    """
    # Filter to only include chunks from relevant articles
    filter_dict = {'article': {'$in': relevant_topic_names}}
    
    # Perform similarity search with the query and filter
    relevant_chunks = chunk_vector_store.similarity_search(query, k=top_k, filter=filter_dict)
    
    # Extract the chunk texts from the search results
    return [doc.page_content for doc in relevant_chunks]

In [32]:
# Load the chunk vector store containing description embeddings
chunk_vector_store = FAISS.load_local('chunk_faiss_index', embeddings, allow_dangerous_deserialization=True)

In [33]:
# Retrieve the relevant chunks
relevant_chunks = retrieve_relevant_chunks(query, relevant_topic_names, chunk_vector_store)

In [37]:
# Print the retrieved chunks
print("Нашли:")
for i, chunk in enumerate(relevant_chunks, 1):
    print(f"{i}. {chunk}")

Нашли:
1. Если поставщик отказался от заключения контракта или истекло время подписания контракта, происходит автоматический отказ от участия поставщика в закупки по потребности.
В результате отказа поставщика от заключения контракта по закупки по потребности происходят следующее:

предложение поставщика исчезает с вкладки «Предложение» закупки по потребности;
заказчику приходит уведомление о смене статуса контракта;

у заказчика появляется возможность выбрать предложение другого поставщика.


 
 
После выбора заказчиком второго поставщика по закупки по потребности происходит стандартный процесс подписания контракта.
2. После отказа от заключения контракта заказчик вместо отправления нового проекта продлил сроки подписания по неактуальному документу. Для решения данного вопроса вам необходимо связаться с заказчиком, для направления нового проекта контракта.


In [ ]:
def local_model_call(user_prompt, retrieved_chunks):
    # Create the prompt with the user's request
    context = "\n".join(retrieved_chunks)
    
    prompt = f"""
Ты - ассистент по поиску информации в данных. Используй следующий контекст для ответа на вопрос пользователя.
Если ты считаешь, что контекста недостаточно для формирования ответа, ответь что не можешь помочь в решении данного вопоса
Контекст:
{context}

Запрос пользователя:
{user_prompt}

Ответ:
"""
    # Define the Ollama API endpoint
    url = "http://localhost:11434/api/generate"
    
    # Prepare the request payload
    data = {
        "model": "yandex/YandexGPT-5-Lite-8B-instruct-GGUF:latest",  # Specify YandexGPT as the model
        "prompt": prompt       # The instruction for YandexGPT
    }
    
    # Send the request to Ollama API with streaming enabled
    response = requests.post(url, json=data, stream=True)
    
    # Collect the generated search query from the streamed response
    search_query = ""
    for line in response.iter_lines():
        if line:
            json_response = json.loads(line)
            if "response" in json_response:
                search_query += json_response["response"]
                print(json_response["response"], end='')
            if json_response.get("done", False):
                break
    
    # Return the cleaned-up search query
    return search_query.strip()

In [ ]:
answer = local_model_call(query, relevant_chunks)

 После отказа поставщика от заключения контракта заказчик может выбрать предложение другого поставщика и начать стандартный процесс подписания контракта. Для этого необходимо:
1. Выбрать новое предложение от другого поставщика.
2. Начать процедуру подписания контракта с новым поставщиком.


'После отказа поставщика от заключения контракта заказчик может выбрать предложение другого поставщика и начать стандартный процесс подписания контракта. Для этого необходимо:\n1. Выбрать новое предложение от другого поставщика.\n2. Начать процедуру подписания контракта с новым поставщиком.'

In [42]:
print(answer)

После отказа поставщика от заключения контракта заказчик может выбрать предложение другого поставщика и начать стандартный процесс подписания контракта. Для этого необходимо:
1. Выбрать новое предложение от другого поставщика.
2. Начать процедуру подписания контракта с новым поставщиком.
